In this block of code it will ask for your token from Hugging Face.

If you don't have an account you can make one here: https://huggingface.co/join

Before you can do this you need to accept the license agreement for the model, which you can do here: https://huggingface.co/CompVis/stable-diffusion-v1-4

On the token page select "new token" and ask for a "read" token (you can name it "Colab").

In [ ]:
!pip install --quiet --upgrade accelerate==0.30.1 peft==0.11.1 transformers==4.41.1 diffusers==0.27.2 scipy==1.13.1 mediapy==1.2.0 networks==0.3.7 utils==1.0.2 pillow==10.3.0
!git clone https://github.com/RandomGamingDev/MCSkinsGen
!huggingface-cli login

In [ ]:
import transformers
import peft
import diffusers
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, StableDiffusionPipeline
import mediapy as media
import torch
from torch import autocast
from safetensors.torch import load_file
import PIL
from PIL import Image
import os

#@markdown Stable Diffusion Model
model_id = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}
#@markdown The Device (this is mostly just for if you're running it on a different machine)
device = "cuda" #@param {type:"string"}
torch.set_default_device(device)

scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True, steps_offset=1)
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, variant="fp16", safety_checker=None, use_auth_token=True).to(device)

#@markdown LoRA Model
lora_model = "mcskins-18.safetensors" #@param {type:"string"}

pipe.load_lora_weights(f"/content/MCSkinsGen/models/{ lora_model }")

# Skin Mask
skin_mask = Image.open('/content/MCSkinsGen/skin_mask.png').convert("RGBA")

Here is where you actually make images. Change the "prompt" to whatever you want to try and then change "num_images" if you want more than one image generated. You can re-run this cell without having to re-run everything before it.

In [ ]:
#@markdown Generation Parameters
skin_name = "WWI Infantry Soldier" #@param {type:"string"}
skin_category = "people" #@param ["movies", "tv", "games", "people", "fantasy", "mobs", "others"]
skin_description = "An infantry soldier from WWI" #@param {type:"string"}
prompt = f"A { skin_name } minecraft skin in the { skin_category } category. { skin_description }"
num_images = 8 #@param {type:"number"}
#@markdown (I recommend generating 8 images once you know what you want)

prompts = [prompt] * num_images

#@markdown AI Parameters
guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
#@markdown Recommended values: <br/>
#@markdown - Most Recommended: `guidance_scale=7.5, num_inference_steps=50` <br/>
#@markdown - `guidance_scale=15, num_inference_steps=50 or 100 or 200` <br/>
#@markdown - `guidance_scale=30, num_inference_steps=100`

with autocast("cuda"):
    images = pipe(prompts, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps).images

#@markdown To download the skins simply right click and press `Save As`
mc_skin_size = (64, 64)
images = [PIL.Image.composite(image.convert("RGBA").resize(mc_skin_size, resample=PIL.Image.Resampling.NEAREST), PIL.Image.new("RGBA", mc_skin_size), skin_mask) for image in images]
media.show_images(images)